In [1]:
import gensim
import nltk
import numpy as np

import load

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

cuda = True
taille_vecteur = 300
hidden_dim = 20
num_layers = 1

In [24]:
X_j = load.load_jokes()
X_b = load.load_bbc()
X_r = load.load_random(10000)

y_j = np.full(len(X_j),0).tolist()
y_b = np.full(len(X_b),1).tolist()
y_r = np.full(len(X_r),2).tolist()

X = [] 
X += X_j
X += X_b
X += X_r

y = [] 
y += y_j
y += y_b
y += y_r

X = np.array(X)
y = np.array(y)

idx = np.arange(len(X))
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

/home/ash/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(9708, 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/ash/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(10549, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/ash/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(10000, 2) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


# Word2vec

In [2]:
# w2v = gensim.models.Word2Vec.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 
w2v = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)
def to_vec(sentence):
    res = np.empty((len(sentence) +1, taille_vecteur)) 
    for i in range(len(sentence)):
        try:
            res[i] = w2v.wv[sentence[i]]
        except KeyError:
            res[i] = w2v.wv["the"]
    res[-1] = np.zeros(taille_vecteur)
    return res

def doc_to_vec(X):
    res = []
    for i in range(len(X)):
        res += [to_vec(X[i])]
    return res

In [25]:
X = doc_to_vec(X)
X_j = doc_to_vec(X_j)

del(X_b)
del(X_r)

# classifieur

In [5]:
class Classifieur(nn.Module):

    def __init__(self, taille_vecteur, hidden_dim, num_layers, cuda=False):
        super(Classifieur, self).__init__()
        self.taille_vecteur = taille_vecteur
        self.hidden_dim = hidden_dim
        
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(taille_vecteur, hidden_dim, num_layers=num_layers)
        self.hidden = self.init_hidden()
        
        self.lin = nn.Linear(hidden_dim , int(hidden_dim/2))
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(int(hidden_dim/2) , 3)
        self.softmax = nn.Softmax(dim=1) 
        
        if cuda:
            self.cuda()
        
        

    def init_hidden(self):
        if self.cuda:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda()),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda()))
        else:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)))


    def forward(self, inputs):    

        out, self.hidden = self.lstm(inputs.view(-1,1,300), self.hidden)
        out = out[-1].view(-1, self.hidden_dim)
        out = self.lin(out)
        out = self.relu(out)
        out = self.lin2(out)
        out = self.softmax(out)
        return out
    
    def prep_label(self, y):
        if self.cuda:
            return autograd.Variable(torch.LongTensor(y).cuda())
        else:
            return autograd.Variable(torch.LongTensor(y))
    
    def train_epoch(self, X, y):


        optimizer = optim.Adam(self.parameters())
        if self.cuda:
            loss_function = nn.CrossEntropyLoss().cuda()
        else:
            loss_function = nn.CrossEntropyLoss()

        loss_cum = 0
        acc = 0

        for j in range(len(X)):

            self.zero_grad()
            self.hidden = self.init_hidden()
            vectorize = X[j]

            if self.cuda:
                tens_X = autograd.Variable(torch.FloatTensor(vectorize).cuda())
            else:
                tens_X = autograd.Variable(torch.FloatTensor(vectorize))


            output = self.forward(tens_X)
            loss = loss_function(output, self.prep_label(np.array(y[j])))

            if self.cuda:
                loss_cum += loss.data.cpu().numpy()
                if np.argmax(output.data.cpu().numpy()) == y[j]:
                    acc += 1
            else:
                loss_cum += loss.data.numpy()
                if np.argmax(output.data.numpy()) == y[j]:
                    acc += 1

            loss.backward()
            optimizer.step()

        return acc/len(X), loss_cum/len(X)

    def train(self, X, y, epoch=10):
        for i in range(epoch):
            print("epoch", i)
            acc , loss = self.train_epoch(X, y)
            print("acc : {}, loss : {}".format(acc, loss))
        return self

# Gan

In [18]:
class Generateur(nn.Module):
    
    def __init__(self, taille_vecteur, hidden_dim, num_layers):
        super(Generateur, self).__init__()
        self.taille_vecteur = taille_vecteur
        self.hidden_dim = hidden_dim
        
        self.num_layers = num_layers
       
        self.lstm = nn.LSTM(taille_vecteur, hidden_dim, num_layers=num_layers)
        self.hidden = self.init_hidden()
        self.lin = nn.Linear(hidden_dim , taille_vecteur)
        

    def init_hidden(self):
        if cuda:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda().float()),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda().float()))
        else:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)))


    def forward(self, inputs):
#         print(inputs.view(-1,1,300))
        out, self.hidden = self.lstm(inputs.view(1,1,-1).float(), self.hidden)
        out = out[-1].view(-1, self.hidden_dim)
        out = self.lin(out)
        return out

In [19]:
class Discriminent(nn.Module):
    
    def __init__(self, taille_vecteur, hidden_dim, num_layers):
        super(Discriminent, self).__init__()
        self.taille_vecteur = taille_vecteur
        self.hidden_dim = hidden_dim
        
        self.num_layers = num_layers

        self.lstm = nn.LSTM(taille_vecteur, hidden_dim, num_layers=num_layers)
        self.hidden = self.init_hidden()
        self.lin = nn.Linear(hidden_dim , 1)
        

    def init_hidden(self):
        if cuda:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda().float()),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda().float()))
        else:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)))


    def forward(self, inputs):

        out, self.hidden = self.lstm(inputs.view(-1,1,300), self.hidden)
        out = out[-1].view(-1, self.hidden_dim)

        out = self.lin(out)
        
        return nn.Sigmoid()(out)

## train

In [20]:
def is_stop(w):
        
        dist = np.linalg.norm(w)

        if dist < 0.2:
            return True
        return False
    
def gen(model, seeds):


    all_blague = []
    if cuda:
        tens_s = autograd.Variable(torch.from_numpy(seeds).float().cuda())
    else:
        tens_s = autograd.Variable(torch.from_numpy(seeds).float())
        
    for j in range(len(seeds)):
        model.hidden = model.init_hidden()

#         print(tens_s[j])
        output =[model(tens_s[j])]
        if cuda:
            res = [output[-1].data.cpu().numpy().reshape(300)]
        else:
            res = [output[-1].data.numpy().reshape(300)]
            
        for i in range(1, 30):
            output += [model(output[i-1].view(300))]
            if cuda:
                res += [output[-1].data.cpu().numpy().reshape(300)]
            else:
                res += [output[-1].data.numpy().reshape(300)]

            if cuda:
                if is_stop(output[-1].data.cpu().numpy().reshape(300)):
                    break
            else:
                if is_stop(output[-1].data.numpy().reshape(300)):
                    break

        all_blague += [res]

    return all_blague

In [21]:
def fit(generateur, discriminent, classifieur, X, y, X_j, epoch=5, d_epoch=10, g_epoch=1, batch=500):
    print("classifieur train")
#     classifieur.train(X, y, epoch=2)
    print("fin train classifieur")
    
    if cuda:
        loss_function = nn.BCELoss().cuda()
        loss_function_oracle = nn.CrossEntropyLoss().cuda() 
    else:
        loss_function = nn.BCELoss()
        loss_function_oracle = nn.CrossEntropyLoss()
        
    opti_G = optim.SGD(generateur.parameters(), lr=0.001)
    opti_D = optim.Adam(discriminent.parameters())
    
    loss_cum_d = np.empty(0)
    loss_cum_g = np.empty(0)
    
    for e in range(epoch):
        print("epoch gan :", e)

        for d_e in range(0, d_epoch, ):
            print("epoch d :", d_e)
            loss_cum = 0
            
            for i in  range(0, len(X_j), batch):
                seeds = np.random.normal(0, 1, (batch, 300))

                d_fake = gen(generateur, seeds)
                d_real = X_j[batch*i: batch*(i+1)].copy()
                
                idx = np.arange(len(d_fake)+len(d_real))
                np.random.shuffle(idx)

                if cuda:
#                     d_all = autograd.Variable(torch.FloatTensor(np.array(d_real+ d_fake)[idx]).cuda())
                    y_all = autograd.Variable(torch.FloatTensor(np.concatenate((np.ones(len(d_real)), np.zeros(len(d_fake))))[idx]).cuda())
                else:
#                     d_all = autograd.Variable(torch.FloatTensor(np.array(d_real+ d_fake)[idx]))
                    y_all = autograd.Variable(torch.FloatTensor(np.concatenate((np.ones(len(d_real)), np.zeros(len(d_fake))))[idx]))
                d_all = np.array(d_real+ d_fake)[idx]
#                 y_all =np.concatenate((np.ones(len(d_real)), np.zeros(len(d_fake))))[idx]
    
                del d_fake
                del d_real
                
                discriminent.zero_grad()

                for i in range(len(d_all)):
                    discriminent.hidden = discriminent.init_hidden()
                    if cuda:
                        inp = autograd.Variable(torch.FloatTensor(np.array(d_all[i])).cuda())
#                         target = autograd.Variable(torch.FloatTensor(y_all[i].reshape(1, 1)).float().cuda())
                    else:
                        inp = autograd.Variable(torch.FloatTensor(np.array(d_all[i])))
#                         target = autograd.Variable(torch.FloatTensor(y_all[i].reshape(1, 1)).float())

                    out = discriminent(inp)
#                     print(out.data.cpu().numpy(), target)
                    loss = loss_function(out, y_all[i])

                    if cuda:
                        loss_cum += loss.data.cpu().numpy()
                    else:
                        loss_cum += loss.data.numpy()

                    if i%batch == 0:
                        loss.backward()
                        opti_D.step()
                        discriminent.zero_grad()


                loss.backward()
                opti_D.step()
            
            loss_cum_d = np.concatenate((loss_cum_d, np.array(loss_cum)) )
                

            del d_all

        for g_e in range(g_epoch):
            print("epoch g :", g_e)
            loss_cum=0
            for j in range(0, len(X_j), batch):
                generateur.zero_grad()
                generateur.hidden = generateur.init_hidden()
                
                seeds = np.random.normal(0, 1, (batch, 300))
                g_fake = gen(generateur, seeds)
            
                for i in range(len(g_fake)):
                    discriminent.hidden = discriminent.init_hidden()
                    
                    if cuda:
                        inp = autograd.Variable(torch.FloatTensor(np.array(g_fake[i])).cuda())
                    else:
                        inp = autograd.Variable(torch.FloatTensor(np.array(g_fake[i])))
                    out = discriminent(inp)

                    if cuda:
                        loss = loss_function(out, autograd.Variable(torch.from_numpy(np.ones(1))).cuda().float())
                    else:
                        loss = loss_function(out, autograd.Variable(torch.from_numpy(np.ones(1))).float())

                    if cuda:
                         loss_cum += loss.data.cpu().numpy()
                    else:

                        loss_cum += loss.data.numpy()

                loss.backward()
                opti_G.step()
            loss_cum_g = np.concatenate((loss_cum_g, np.array(loss_cum)) )
            
        del g_fake
        
    return loss_cum_d, loss_cum_g

In [10]:
classifieur = Classifieur(taille_vecteur, hidden_dim, num_layers, cuda).cuda()
generateur = Generateur(taille_vecteur, hidden_dim, num_layers).cuda()
discriminent = Discriminent(taille_vecteur, hidden_dim, num_layers).cuda()

In [27]:
loss_d, loss_g = fit(g, d, None, X, y, X_j, epoch=4, d_epoch=0, g_epoch=2)

classifieur train
fin train classifieur
epoch gan : 0
epoch g : 0


/home/ash/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/home/ash/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1360: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


epoch g : 1
epoch gan : 1
epoch g : 0
epoch g : 1
epoch gan : 2
epoch g : 0
epoch g : 1
epoch gan : 3
epoch g : 0
epoch g : 1


In [13]:
print(loss_d)

[ 4638.30371094  1273.52929688   780.14007568   592.40264893   475.76641846
   392.38818359   311.57058716   256.90881348   217.20759583   177.59469604
   153.33242798   124.49014282   109.13217926    93.04379272    82.70018005
    71.48114777    61.78045273    56.10108948    51.69104004    49.18974686
    44.51403427    38.18915558    34.07662964    30.67892647    28.01355553
    26.0185833     23.48913956    22.04730988    20.44248581    19.55508804
    18.07347679    16.62158775    15.74840736    14.76042843    13.85342216
    12.91252613    12.31938362    11.66024399    11.14214993    10.56401062]


In [14]:
print(loss_g)

[ 49814.9375    60585.953125  67422.421875  72423.515625]


In [16]:
seeds = np.random.normal(0, 1, (15, 300))
g_fake = gen(generateur, seeds)

In [ ]:
for j in range(len(g_fake)):
    for i in range(len(g_fake[j])):
        print(w2v.most_similar([g_fake[j][i]])[0][0], end=" ")
    print("")